In [1]:
import numpy as np

from GRAPE import *
import matplotlib.pyplot as plt
from qiskit import Aer, execute

In [2]:
J = np.zeros((3, 3))
J[0][1] = 0.1385
J[1][2] = 0.01304
J[0][2] = 0.00148

CX = np.eye(4)
CX[2][2] = 0
CX[3][3] = 0
CX[2][3] = 1
CX[3][2] = 1

SWAP = np.eye(4)
SWAP[1][1] = 0
SWAP[2][2] = 0
SWAP[1][2] = 1
SWAP[2][1] = 1

TOFFOLI = np.eye(8)
TOFFOLI[6][6] = 0
TOFFOLI[7][7] = 0
TOFFOLI[6][7] = 1
TOFFOLI[7][6] = 1

In [3]:
n = 1
m = 1000
k = 4

descents = [GradientDescend(CX, k) for _ in range(n)]

for descent in descents:
    descent.noise = 0.05
    descent.randomize_params()
    descent.set_j(J)

losses = [descent.descend(steps=m, track_distance=True) for descent in descents]

for loss in losses:
    plt.plot(np.real(loss))
for i in range(n):
    print("distance:", descents[i].distance.round(6).real, "; time:", descents[i].time.round(2))

C:\Users\Panfe\PycharmProjects\GRAPE\GRAPE.py:95: ComplexWarning: Casting complex values to real discards the imaginary part
  evolution = evolution @ (math.cos(math.pi / 2 * self.J[i][j] * self.time) * np.eye(2 ** self.size,
C:\Users\Panfe\PycharmProjects\GRAPE\GRAPE.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  dtype=complex) - 1j * math.sin(


AttributeError: 'Evolution' object has no attribute '_Evolution__Z'

In [ ]:
for i in range(n):
    for j in range(k):
        print(descents[i].gates[j].time, end=" ")
    print("\n")

circuit = descents[0].to_qiskit()

backend = Aer.get_backend('unitary_simulator')
job = execute(circuit, backend)
result = job.result()
matrix = result.get_unitary(circuit, decimals=2)
print(descents[0].matrix.round(2), "\n" ,matrix.round(2))

circuit.draw()
